In [5]:
import os
import pandas as pd

def exportar_condition_para_modeled(diretorio_curated, diretorio_modeled, arquivo_saida, chunksize):
    arquivos_parquet = [f for f in os.listdir(diretorio_curated) if f.startswith('lote_') and f.endswith('.parquet')]
    
    if not arquivos_parquet:
        print("🚨 Nenhum arquivo Parquet encontrado no diretório.")
        return
    
    os.makedirs(diretorio_modeled, exist_ok=True)
    
    lista_conditions = []

    for arquivo in arquivos_parquet:
        caminho_arquivo = os.path.join(diretorio_curated, arquivo)
        print(f"🔄 Lendo arquivo: {caminho_arquivo}")

        # ✅ Lê o arquivo inteiro (sem chunksize)
        df = pd.read_parquet(caminho_arquivo, engine='pyarrow')

        # ✅ Divide em chunks manualmente
        total_registros = len(df)
        for i in range(0, total_registros, chunksize):
            chunk = df.iloc[i:i + chunksize].reset_index(drop=True)

            # ✅ Filtrar as entradas onde o resourceType é 'Condition'
            df_condition = chunk[chunk['resource_resourceType'] == 'Condition']
            
            # ✅ Agrupar por 'resource_code_text' e contar as ocorrências
            condition_counts = (df_condition
                                .groupby('resource_code_text')
                                .size()
                                .reset_index(name='count'))
            
            lista_conditions.append(condition_counts)

    # ✅ Concatena todos os lotes
    if lista_conditions:
        df_final = pd.concat(lista_conditions, ignore_index=True)
        
        # ✅ Agrupar novamente para consolidar os resultados finais
        df_final = (df_final
                    .groupby('resource_code_text', as_index=False)
                    .sum()
                    .sort_values(by='count', ascending=False)
                    .head(10))
        
        # ✅ Salva como Excel
        caminho_saida = os.path.join(diretorio_modeled, arquivo_saida)
        df_final.to_excel(caminho_saida, index=False)
        print(f"✅ Arquivo de saída salvo em: {caminho_saida}")
    else:
        print("🚨 Nenhum dado de 'Condition' encontrado para exportar.")

# 🔥 Exemplo de uso:
diretorio_curated = r'C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\condition'
diretorio_modeled = r'C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\modeled'
arquivo_saida = 'top_conditions.xlsx'
chunksize = 1000

# ✅ Executa o processo de exportação
exportar_condition_para_modeled(diretorio_curated, diretorio_modeled, arquivo_saida, chunksize)

🔄 Lendo arquivo: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\condition\lote_001.parquet
🔄 Lendo arquivo: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\condition\lote_002.parquet
🔄 Lendo arquivo: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\condition\lote_003.parquet
🔄 Lendo arquivo: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\condition\lote_004.parquet
🔄 Lendo arquivo: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\condition\lote_005.parquet
🔄 Lendo arquivo: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\condition\lote_006.parquet
🔄 Lendo arquivo: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\condition\lote_007.parquet
🔄 Lendo arquivo: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\condition\lote_008.parquet
🔄 Lendo arquivo: C:\Users\Leonar

In [49]:
import glob

arquivos = glob.glob(r'C:/Users/Leonardo/OneDrive/Documentos/Git/teste_engenheiro/eng_dados/raw/*.parquet')
df = pd.concat([pd.read_parquet(f) for f in arquivos], ignore_index=True)

In [52]:
for col in df.columns:
    print(col)

fullUrl
resource_resourceType
resource_id
resource_text_status
resource_text_div
resource_extension
resource_identifier
resource_name
resource_telecom
resource_gender
resource_birthDate
resource_address
resource_maritalStatus_coding
resource_maritalStatus_text
resource_multipleBirthBoolean
resource_communication
request_method
request_url
resource_active
resource_type
resource_status
resource_class_system
resource_class_code
resource_subject_reference
resource_subject_display
resource_participant
resource_period_start
resource_period_end
resource_serviceProvider_reference
resource_serviceProvider_display
resource_clinicalStatus_coding
resource_verificationStatus_coding
resource_code_coding
resource_code_text
resource_encounter_reference
resource_onsetDateTime
resource_recordedDate
resource_udiCarrier
resource_distinctIdentifier
resource_manufactureDate
resource_expirationDate
resource_lotNumber
resource_serialNumber
resource_deviceName
resource_type_coding
resource_type_text
resource_p